In [1]:
import pandas as pd

airline = pd.read_csv("airline.txt")
airline['CONS']=1
airline.head()

,ARR_DELAY,DAY_OF_WEEK,DEP_DELAY,DISTANCE,CONS
0,-2,5,3,31,1
1,-20,6,0,67,1
2,-17,2,-18,67,1
3,-17,1,-2,67,1
4,-14,2,-9,67,1


In [2]:
day_dummies = pd.get_dummies(airline["DAY_OF_WEEK"],prefix='DAY')
day_dummies = day_dummies.drop('DAY_7',axis=1)
day_dummies.head()

,DAY_1,DAY_2,DAY_3,DAY_4,DAY_5,DAY_6
0,0,0,0,0,1,0
1,0,0,0,0,0,1
2,0,1,0,0,0,0
3,1,0,0,0,0,0
4,0,1,0,0,0,0


In [3]:
X = pd.concat([airline[['CONS','DISTANCE','DEP_DELAY']],day_dummies],axis=1)
Y = airline['ARR_DELAY']

In [4]:
import numpy as np

b_ols = np.matmul(np.linalg.inv(np.matmul(np.transpose(X),X)),np.matmul(np.transpose(X),Y))
print(b_ols)

[-1.26705555 -0.00313306  1.01656564  0.84352555 -0.07468387  1.01286906
  0.50906635 -0.87298581 -0.69730066]


In [5]:
import scipy as sp
from scipy.optimize import fmin

def f(b):
    return np.sum((Y-b[0]*X['CONS']-b[1]*X['DISTANCE']-b[2]*X['DEP_DELAY']-b[3]*X['DAY_1']-b[4]*X['DAY_2']
            + -b[5]*X['DAY_3']-b[6]*X['DAY_4']-b[7]*X['DAY_5']-b[8]*X['DAY_6'])**2)

x0 = np.array([0,0,0,0,0,0,0,0,0])
betas = fmin(f,x0,xtol=1e-8,maxfun=10000)
betas

Optimization terminated successfully.
         Current function value: 4325130.053045
         Iterations: 2583
         Function evaluations: 3816


array([-0.94017593, -0.00337522,  1.01578634,  1.07916508, -0.54383536,
        0.88797133,  0.57443649, -0.69911015, -0.71756321])

In [8]:
from scipy.optimize import basinhopping

betas2 = basinhopping(f,x0,niter=10)
betas2.x

array([-1.26704996, -0.00313308,  1.01656564,  0.84353371, -0.07467575,
        1.01287327,  0.50907891, -0.87297565, -0.69729074])